# Engaging Image Input/Output with OpenAI Responses API in AG2

**Authors:** Yixuan Zhai

This notebook demonstrates how to do image input and image generate through a two agent chat with OpenAI Responses API and their GPT-4o model.

### Set LLM config to use OpenAI response API

In [ ]:
import base64
import os
import textwrap

from autogen import AssistantAgent, UserProxyAgent

# ------------------------------------------------------------------ LLM config
llm_cfg = {
    "config_list": [
        {
            "api_type": "responses",  # use /responses
            "model": "gpt-4o",  # supports vision + images
            "api_key": os.getenv("OPENAI_API_KEY"),
        }
    ]
}

### Create an assistant for image processing

In [ ]:
assistant = AssistantAgent(
    name="ArtBot",
    llm_config=llm_cfg,
    system_message=textwrap.dedent("""
        You are an assistant that can reason over images and
        use the built-in image_generation tool. When generating
        an image, return ONLY the tool call result you receive.
    """).strip(),
)

#  initial image (URL or data-URI)
IMAGE_URL = "https://upload.wikimedia.org/wikipedia/commons/3/3b/BlkStdSchnauzer2.jpg"

### Run chat

Run the following cell and ask two follow up questions, separately, to have two new images created.
1. Generate a watercolor painting of the image.
2. Could you update the image to use Ghibli style?

In [ ]:
response = assistant.run(
    message=f"Describe this image <{IMAGE_URL}> in one sentence",
    user_input=True
)

response.process()

### Save generated images

Run the following cell to save the generated images from the conversation. An image will be saved for each message in the chat history that had an image generated.

In [ ]:
# ----helper function to save image from base64 string----
def save_b64_png(b64_str, fname="generated.png"):
    with open(fname, "wb") as f:
        f.write(base64.b64decode(b64_str))
    print(f"image saved → {fname}")

messages = response.messages
for i in range(len(messages)):
    print(i)
    message = messages[i]
    # print(message)
    if message.get("name") == "ArtBot":
        contents = message.get("content", [])
        for content in contents:
            if (
                content.get("type") == "tool_call"
                and content.get("name") == "image_generation"
                and "content" in content
                and content["content"]
            ):
                print("Saving image!")
                save_b64_png(content["content"], f"image{i}.png")